In [9]:
import nltk
# nltk.download('punkt', download_dir='NLTK_DATA')
# nltk.download('stopwords', download_dir='NLTK_DATA')
nltk.data.path.append("NLTK_DATA")
import pandas as pd
import numpy as np
from processing import freqs, normalisasi
from sentiment import *

# PREPARASI *TRAINING DATASET*

In [10]:
# TODO: pindah ke sqlite
dataset = pd.read_csv("dataset/training_dataset.csv")
all_pos = dataset[dataset['label'] == 1]
all_neg = dataset[dataset['label'] == 0]

# * Training data: 80% pertama
# * Testing data: 20% terakhir
train_pos = all_pos[:int(len(all_pos)*0.8)]
train_neg = all_neg[:int(len(all_neg)*0.8)]
test_pos = all_pos[-int(len(all_pos)*0.2):]
test_neg = all_neg[-int(len(all_neg)*0.2):]

train_x = train_pos.append(train_neg, ignore_index=True)
test_x = test_pos.append(test_neg, ignore_index=True)

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

/tmp/ipykernel_15584/3910910574.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_x = train_pos.append(train_neg, ignore_index=True)
/tmp/ipykernel_15584/3910910574.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_x = test_pos.append(test_neg, ignore_index=True)


In [27]:
ld = len(dataset)
trp = len(train_pos)
tng = len(train_neg)
tsp = len(test_pos)
tsn = len(test_neg)
print(f"dataset: {ld} \n train pos: {trp} \n train neg: {tng} \n test pos: {tsp} \n test neg: {tsn}")

dataset: 1300 
 train pos: 511 
 train neg: 528 
 test pos: 127 
 test neg: 132


In [29]:
len(all_pos)

639

# PELATIHAN *MODEL CLASSIFIER*

In [12]:
x = train_x['text']
y = train_y
kamus_frek = kamus_freq(x, y)

In [ ]:
for x in kamus_frek.keys():
    print(kamus_frek.get(x, 0))

## logprior
$D$ = jumlah dokumen <br>
$D_{pos}$ = jumlah dokumen positif <br>
$D_{neg}$ = jumlah dokumen negatif <br>
### probabilitas dokumen positif
$P(D_{pos}) = \frac{D_{pos}}{D}$
### probabilitas dokumen negatif
$P(D_{neg}) = \frac{D_{neg}}{D}$
### probabilitas prior
${logprior} = log \left( \frac{P(D_{pos})}{P(D_{neg})} \right) = log \left( \frac{D_{pos}}{D_{neg}} \right)$
<br> disederhanakan, menjadi: <br>
$logprior = P(D_{pos}) - P(D_{neg})$

## loglikelihood
$W$ = probabilitas positif/negatif kata <br>
$freq$ = frekuensi kata unik yang bernilai positif/negatif <br>
$N$ =  jumlah total kata positif/negatif <br>
$V$ = jumlah total kata unik (panjang dokumen frekuensi) <br>
### probabilitas kata positif
$P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}$
### probabilitas kata negatif
$P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}$
### likelihood kata
$\text{loglikelihood} = \log \left(\frac{P(W_{pos})}{P(W_{neg})} \right)$

In [13]:
logprior, loglikelihood = train_nbc(kamus_frek, train_y)

## HASIL PENGUJIAN MODEL
- data *training*
- data *testing*

In [ ]:
test = test_nbc(test_x['text'], test_y, logprior, loglikelihood)
test

## UJI KESELURUHAN *DATASET*

In [ ]:
data_test = test_nbc(dataset['text'], dataset['label'], logprior, loglikelihood)
data_test

## *CONFUSION MATRIX*

## metrik-metrik
-   TP = True Positive
-   TN = True Negative
-   FP = False Positive
-   FN = False Negative
- Populasi = TP+TN+FP+FN
<hr>

### akurasi

jumlah prediksi yang benar. tidak dianjurkan sebagai metrik kunci. <br>
$\frac{TP+TN}{Populasi}$

### miss

jumlah prediksi yang salah <br>
$\frac{FP+FN}{Populasi}$

### presisi

persentase positif yang diprediksi secara akurat. jika FP = 0, maka nilai presisi 100%. <br>
$\frac{TP}{TP+FP}$

### sensitifitas

nilai positif sesungguhnya <br>
$\frac{TP}{TP+FN}$

In [14]:
pdc = pd.DataFrame()
pdc = dataset
pdc['label_prediksi'] = pdc['text'].apply(lambda x: predict_nbc(x, logprior, loglikelihood))

In [15]:
# TODO: Do more testing with thresholds values from 0, 0.5, 0.8, 0.9, 1
# * to note;
# treshold = -0.03272669350255697
# * default treshold is 0
def toLabel(x):
   if x > 0:
       return 1
   elif x < 0:
       return 0
   else:
       return "n"
pdc['prediksi'] = pdc['label_prediksi'].apply(lambda x: toLabel(x))

In [16]:
pdc

,text,label,label_prediksi,prediksi
0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,0,-29.162351,0
1,Geblek lo tata...cowo bgt dibela2in balikan......,0,-7.665502,0
2,Kmrn termewek2 skr lengket lg duhhh kok labil ...,0,-10.987756,0
3,"Intinya kalau kesel dengan ATT nya, gausah ke ...",0,-27.500103,0
4,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",0,-11.574785,0
...,...,...,...,...
1295,Fargo juga adaptasi dari film yang cukup berha...,1,-0.014864,0
1296,637.000 waw ini sangat keren flm horor dng jum...,1,3.765399,1
1297,@filmziarah film yang tenang dan menghanyutkan...,1,-0.278294,0
1298,Film yg amat menarik. Kisah cinta & kesetiaan ...,1,3.855586,1


In [ ]:
tr = logprior
def toLabel(x):
   if x > tr:
       return 1
   elif x < tr:
       return 0
   else:
       return "n"
pdc['prediksi_tr'] = pdc['label_prediksi'].apply(lambda x: toLabel(x))

In [17]:
conf_matrix = pd.crosstab(pdc['label'], pdc['prediksi'], rownames=['Actual'], colnames=['Predicted'])
conf_matrix

Predicted,0,1
Actual,,
0,647,14
1,84,555


In [19]:
pop = 647+555+14+84
a = (647+555)/pop
m = (14+84)/pop
p = 555/(555+14)
s = 555/(555+84)
print(f"pop={pop}\nakurasi:{a}, miss:{m}, presisi: {p}, sensi: {s}.\n F1:{2*(p*s)/(p+s)}")

pop=1300
akurasi:0.9246153846153846, miss:0.07538461538461538, presisi: 0.9753954305799648, sensi: 0.8685446009389671.
 F1:0.9188741721854304


In [45]:
pdc

,text,label,label_prediksi,prediksi,label_akurat
0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,0,-29.162351,0,False
1,Geblek lo tata...cowo bgt dibela2in balikan......,0,-7.665502,0,False
2,Kmrn termewek2 skr lengket lg duhhh kok labil ...,0,-10.987756,0,False
3,"Intinya kalau kesel dengan ATT nya, gausah ke ...",0,-27.500103,0,False
4,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",0,-11.574785,0,False
...,...,...,...,...,...
1295,Fargo juga adaptasi dari film yang cukup berha...,1,-0.014864,0,False
1296,637.000 waw ini sangat keren flm horor dng jum...,1,3.765399,1,False
1297,@filmziarah film yang tenang dan menghanyutkan...,1,-0.278294,0,False
1298,Film yg amat menarik. Kisah cinta & kesetiaan ...,1,3.855586,1,False


In [50]:
acc = 0
for index, row in pdc.iterrows():
    if row['prediksi'] == row['label']:
        acc+=1

acc

1202

In [ ]:
conf2 = pd.crosstab(pdc['label'], pdc['prediksi_tr'], rownames=['Actual'], colnames=['Predicted'])
conf2

In [ ]:
a2 = (647+556)/1300
m2 = (14+83)/1300
p2 = 556/(556+14)
s2 = 556/(556+83)
print(f"akurasi:{a2}, miss:{m2}, presisi: {p2}, sensi: {s2}.\n F1:{2*(p2*s2)/(p2+s2)}")

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

sn.heatmap(conf_matrix, annot=True, fmt='g')
plt.show()

In [ ]:
sn.heatmap(conf2, annot=True, fmt='g')
plt.show()

# UJI CLASSIFIER PADA ARTIKEL BERITA

In [82]:
berita = open('berita.txt', 'r')
berita = berita.read()

In [83]:
beritanormal = normalisasi(berita)
frekberita = freqs(beritanormal)
# frekberita

## SENTIMEN ARTIKEL KESELURUHAN

In [84]:
predict_nbc(berita, logprior, loglikelihood)

-2.6912482077738114

## SENTIMEN ARTIKEL PER_KALIMAT

In [117]:
from nltk.tokenize import sent_tokenize
x = pd.DataFrame()
kalimat = sent_tokenize(berita)
x['kalimat'] = kalimat
x['kalimat_normal'] = x['kalimat'].apply(lambda x: normalisasi(x))

In [118]:
x['label_prediksi'] = x['kalimat'].apply(lambda x: predict_nbc(x, logprior, loglikelihood))

In [119]:
x['prediksi'] = x['label_prediksi'].apply(lambda x: toLabel(x))
x

,kalimat,kalimat_normal,label_prediksi,prediksi
0,Saya pribadi sangat menyukai performa versi 20...,"[pribadi, suka, performa, versi, asa, stabil]",-0.419612,0
1,"Dan, hampir tidak ada masalah berarti yang say...","[arti, jumpa, sistem, operasi]",-4.239262,0
2,Banyak fitur baru yang sangat berguna.,[fitur],-0.032727,0
3,"Tapi buat saya, fitur-fitur tersebut tidak sep...","[fiturfitur, sepenuh, butuh]",-0.950725,0
4,Setelah merasakan enaknya pakai versi 21.10 sa...,"[enak, pakai, versi, enggan, pindah, lts]",1.753910,1
5,"Tapi, saya tetap harus beralih ke versi 21.04 ...","[alih, versi, dukung, juli]",0.371031,1
6,"Selama beberapa bulan menggunakan OS ini, saya...","[os, puas]",-0.034434,0
7,Hampir tak ada kendala berarti yang saya jumpai.,"[kendala, arti, jumpa]",-2.628116,0
8,Semua software yang biasa saya butuhkan bisa d...,"[software, butuh, diinstal, jalan, lancar, ken...",1.107846,1
9,"Mulai dari Google Chrome, WPS Office, Wine dan...","[google, chrome, wps, office, wine, playonlinu...",0.655298,1


## SENTIMEN ARTIKEL PER_KATA

In [88]:
berita_normal = normalisasi(berita)

In [122]:
# * predict_nbc per-kata
def predict_perkata(text, logpri, loglik):
    l = []
    for w in text:
        x = predict_nbc(w, logpri, loglik)
        l.append({
            "kata":w,
            "skor":x
            })
    return l

In [123]:
hasil = predict_perkata(berita_normal, logprior, loglikelihood)
type(hasil[0])

dict

In [124]:
hasil

[{'kata': 'pribadi', 'skor': -0.034434149808107085},
 {'kata': 'suka', 'skor': 0.566339710620823},
 {'kata': 'performa', 'skor': -0.03272669350255697},
 {'kata': 'versi', 'skor': -0.03272669350255697},
 {'kata': 'asa', 'skor': -0.7275813303680525},
 {'kata': 'stabil', 'skor': -0.3221162222598881},
 {'kata': 'arti', 'skor': -1.2384069541340432},
 {'kata': 'jumpa', 'skor': -0.7275813303680526},
 {'kata': 'sistem', 'skor': -1.6438720622422076},
 {'kata': 'operasi', 'skor': -0.7275813303680526},
 {'kata': 'fitur', 'skor': -0.03272669350255697},
 {'kata': 'fiturfitur', 'skor': -0.03272669350255697},
 {'kata': 'sepenuh', 'skor': -0.03272669350255697},
 {'kata': 'butuh', 'skor': -0.9507248816822622},
 {'kata': 'enak', 'skor': 0.1478874069858475},
 {'kata': 'pakai', 'skor': -0.034434149808107085},
 {'kata': 'versi', 'skor': -0.03272669350255697},
 {'kata': 'enggan', 'skor': -0.03272669350255697},
 {'kata': 'pindah', 'skor': 1.5750037626259932},
 {'kata': 'lts', 'skor': -0.03272669350255697},
 

In [129]:
logprior

-0.03272669350255697

In [128]:
import csv
keys = hasil[0].keys()

with open('demo-kata.csv', 'w', newline='') as file:
    dict_writer = csv.DictWriter(file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(hasil)